# Model Entailment 1

This is the first of several notebooks to explore logical entailments in SysML v2 models, as well as the means by which to encode them.

In [ ]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import build_from_classifier_pattern, \
                                    new_element_ownership_pattern, \
                                    build_from_binary_relationship_pattern, \
                                    build_superset_classifier, \
                                    create_new_feature

from uuid import uuid4

## Load up and explore basic model

Load up a basic model in order to have basic package and namespace into which to add additional elements.

In [ ]:
filename = "Model_Loader_Test_Level_2"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()).parent.parent / "tests/fixtures" / filename

level2 = pm.Model.load_from_post_file(json_file)
level2

In [ ]:
len(level2.elements)

In [ ]:
classifiers = [ele for ele in level2.elements.values() if ele._metatype == 'Classifier']
classifiers

Locate the root package of the model to which new elements will be added.

In [ ]:
base_package = classifiers[0].owningRelationship.owningRelatedElement
base_package

## Example Application of Reasoning 1 - Build Common Class from Examples

In this example, we look at a series of specific examples of an item, 4 bicycle wheels.

First, add these elements to the model.

In [ ]:
bw1 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #1", model=level2, specific_fields={}, metatype="Classifier")
bw2 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #2", model=level2, specific_fields={}, metatype="Classifier")
bw3 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #3", model=level2, specific_fields={}, metatype="Classifier")
bw4 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #4", model=level2, specific_fields={}, metatype="Classifier")

In [ ]:
bw1.owningRelationship

Now, create a new superclass that all of these specific wheels can be grouped into.

In [ ]:
new_wheel = build_superset_classifier(classes=[bw1, bw2, bw3, bw4],
                                    super_name="Bike Wheel",
                                    model=level2,
                                    owner=base_package,
                                    added_fields={})

Check that the wheel has its specific versions.

In [ ]:
new_wheel.reverseSubclassification

In [ ]:
base_package.ownedMember

## Developing unrolling rule around connectors

Rule 1 - find connectors with ends that have a multiplicity of 1 and then specialize them.

In [ ]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']

In [ ]:
efms = [ele for ele in level2.elements.values() if ele._metatype == 'EndFeatureMembership']
efms

In [ ]:
for end_feature in connectors[0].throughEndFeatureMembership:
    print(end_feature)
    if 'throughReferenceSubsetting' in end_feature._derived:
        print(f"Feature references {end_feature.throughReferenceSubsetting[0]}")

In [ ]:
features = [ele for ele in level2.elements.values() if ele._metatype == 'Feature']
features

Look for multiplicity ranges in the model that are set to 1.

In [ ]:
def is_type_undefined_mult(type_ele):
    if not 'throughOwningMembership' in type_ele._derived:
        return True
    mult_range = [mr for mr in type_ele.throughOwningMembership if mr['@type'] == 'MultiplicityRange']
    return len(mult_range) == 0

In [ ]:
def is_multiplicity_one(type_ele):
    if not 'throughOwningMembership' in type_ele._derived:
        return False
    multiplicity_range = [mr for mr in type_ele.throughOwningMembership if mr['@type'] == 'MultiplicityRange'][0]
    literal_value = [li.value for li in multiplicity_range.throughOwningMembership if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] == 1
    elif len(literal_value) == 2:
        return literal_value[0] == 1 and literal_value[1] == 1

In [ ]:
def is_multiplicity_specific_finite(type_ele):
    if not 'throughOwningMembership' in type_ele._derived:
        return False
    multiplicity_range = [mr for mr in type_ele.throughOwningMembership if mr['@type'] == 'MultiplicityRange'][0]
    literal_value = [li.value for li in multiplicity_range.throughOwningMembership if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] > 1
    elif len(literal_value) == 2:
        return literal_value[0] > 1 and literal_value[0] == literal_value[1]

In [ ]:
[is_type_undefined_mult(ft) for ft in features]

In [ ]:
[is_multiplicity_one(ft) for ft in features]

In [ ]:
[is_multiplicity_specific_finite(ft) for ft in features]

Find all types (classifiers and features) that have a declared multiplicity that is finite.

In [ ]:
def get_finite_multiplicity_types(model):
    model_types = [ele for ele in model.elements.values() if ele._metatype in ('Feature', 'Classifier')]
    
    return [finite_type for finite_type in model_types if
            is_multiplicity_one(finite_type) or is_multiplicity_specific_finite(finite_type)]

In [ ]:
get_finite_multiplicity_types(level2)

Find where ends the connection is bound to other features in the model.

In [ ]:
refsubs = [ele for ele in level2.elements.values() if ele._metatype == 'ReferenceSubsetting']
refsubs

In [ ]:
def identify_connectors_one_side(connectors):
    one_sided = []
    for connector in connectors:
        if 'throughEndFeatureMembership' in connector._derived:
            for end_feature in connector.throughEndFeatureMembership:

                if 'throughReferenceSubsetting' in end_feature._derived:
                    if is_multiplicity_one(end_feature.throughReferenceSubsetting[0]) and connector not in one_sided:
                        one_sided.append(connector)
    
    return one_sided

In [ ]:
conns_to_cover = identify_connectors_one_side(connectors)
conns_to_cover

In [ ]:
conns_to_cover[0].source

The function below shows the creation of a connector which requires many elements to be created (the connection itself, the ends, references out to other Features in the mind, specialization relationship.

In [ ]:
def create_specific_connector(owner: Element,
                              base_connector: Element,
                              name: str,
                              model: Model,
                              bound_from: Element,
                              bound_to: Element,
                              metatype: str = "Connector"):
    
    # create new connector
    
    connector_dict = {
        'source': [{'@id': bound_from._id}],
        'target': [{'@id': bound_to._id}],
        'ownedRelationship': [],
        'owningRelatedElement': {'@id': '79a99283-cb17-47d2-b77f-81e790ea0e61'}
    }
    
    spec_connector = create_new_classifier(
        owner=owner,
        name=name,
        model=model,
        metatype="Connector",
        added_fields=connector_dict
    )
    
    # specialize the old connector
    
    subclass_added_data = {
         'specific': {'@id': spec_connector._id},
         'general': {'@id': base_connector._id},
         'subclassifier': {'@id': spec_connector._id},
         'superclassifier': {'@id': base_connector._id}
    }

    new_sc = create_new_relationship(
        source=spec_connector,
        target=base_connector,
        owner=spec_connector,
        model=model,
        metatype='Subclassification',
        owned_related_element=None,
        owning_related_element=spec_connector,
        added_fields=subclass_added_data
    )
    
    # specialize the end features to point to the new ends - need to modify library to 
    # be able to pass down preferred membership type
    
    new_bound_from = create_new_feature(owner=spec_connector,
                   name=bound_from.declaredName + ' end (Closed)',
                   model=model,
                   added_fields={},
                   metatype="Feature",
                   member_kind="EndFeatureMembership")
    
    new_bound_from_ref_data = {
        'subsettingFeature': {'@id': new_bound_from._id},
        'subsettedFeature': {'@id': bound_from._id}
    }
    
    new_bound_from_ref = create_new_relationship(
        source=new_bound_from,
        target=bound_from,
        owner=new_bound_from,
        model=model,
        metatype='ReferenceSubsetting',
        owned_related_element=None,
        owning_related_element=new_bound_from,
        added_fields=new_bound_from_ref_data
    )
    
    new_bound_to = create_new_feature(owner=spec_connector,
                   name=bound_to.declaredName + ' end (Closed)',
                   model=model,
                   added_fields={},
                   metatype="Feature",
                   member_kind="EndFeatureMembership")
    
    new_bound_to_ref_data = {
        'subsettingFeature': {'@id': new_bound_to._id},
        'subsettedFeature': {'@id': bound_to._id}
    }
    
    new_bound_to_ref = create_new_relationship(
        source=new_bound_to,
        target=bound_to,
        owner=new_bound_to,
        model=model,
        metatype='ReferenceSubsetting',
        owned_related_element=None,
        owning_related_element=new_bound_to,
        added_fields=new_bound_to_ref_data
    )
    
    return spec_connector

Get the root package again for the new connections to be owned by.

In [ ]:
top_elements = [ele for ele in level2.ownedElement if ele._metatype == "Namespace"][0].throughOwningMembership
top_package = [ele for ele in top_elements if ele._metatype == "Package"][0]
top_package

In [ ]:
for conn in conns_to_cover:
    create_specific_connector(
        owner=top_package,
        base_connector=conn,
        name=conn.declaredName,
        model=level2,
        bound_from=conn.source[0],
        bound_to=conn.target[0],
        metatype="Connector"
    )
    

In [ ]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']
connectors

In [ ]:
conns_to_cover[0].reverseSubclassification

In [ ]:
model_types = [ele for ele in level2.elements.values() if ele._metatype in ('Feature', 'Classifier')]
new_end_1 = [ele for ele in level2.elements.values() if 'declaredName' in ele._data and ele.declaredName == 'Side 1 end (Closed)'][0]
model_types

In [ ]:
new_end_1

Check that the new end is properly linked to the connection and to the Feature to which it was supposed to connect.

In [ ]:
new_end_1.throughReferenceSubsetting

In [ ]:
new_end_1.reverseEndFeatureMembership